In [1]:
from nba_api.live.nba.endpoints import scoreboard
import json


In [7]:
def get_todays_completed_games():
    # Today's Score Board
    games = scoreboard.ScoreBoard()
    live_games_data = games.get_dict()
    live_games_data['scoreboard']
    games = live_games_data['scoreboard']['games']

    print(f"Number of total games : {len(games)}")
    # display(games)

    print("All of Today's Games")
    for i, game in enumerate(games):
        print(
            i, game['gameId'], game['homeTeam']['teamTricode'], game['awayTeam']['teamTricode'], 
            game['gameStatus'],
            game['gameStatusText']
        )

    print("\nGampes Completed")
    finished_games = []
    game_keys = ['gameId']
    for i, game in enumerate(games):
        if game['gameStatus'] == 3:
            print(i, game['gameId'], game['homeTeam']['teamTricode'], game['awayTeam']['teamTricode'])
            finished_games.append({
                'gameId' : game['gameId'],
                'homeTeamTricode' : game['homeTeam']['teamTricode'],
                'awayTeamTricode' : game['awayTeam']['teamTricode'], 
            })

    return finished_games

In [8]:
games = get_todays_completed_games()

Number of total games : 12
All of Today's Games
0 0022301070 SAS NYK 3 Final
1 0022301073 SAC DAL 2 Half
2 0022301071 DEN MIN 2 Q4 4:39
3 0022301062 CHA GSW 3 Final
4 0022301063 IND LAL 3 Final
5 0022301064 ORL LAC 3 Final
6 0022301065 WAS DET 3 Final
7 0022301066 BKN CHI 3 Final
8 0022301067 CLE PHI 3 Final
9 0022301068 MIA POR 3 Final               
10 0022301069 OKC PHX 3 Final
11 0022301072 UTA HOU 2 2nd Qtr             

Gampes Completed
0 0022301070 SAS NYK
3 0022301062 CHA GSW
4 0022301063 IND LAL
5 0022301064 ORL LAC
6 0022301065 WAS DET
7 0022301066 BKN CHI
8 0022301067 CLE PHI
9 0022301068 MIA POR
10 0022301069 OKC PHX


In [12]:
from nba_api.stats.endpoints import playbyplayv2
import datetime

In [23]:
# Function to convert PCTIMESTRING to timedelta
def convert_to_timedelta(time_string):
    minutes, seconds = map(int, time_string.split(':'))
    return datetime.timedelta(minutes=minutes, seconds=seconds)

def time_last_two_minutes(game_id):
    # Game ID for the game you're interested in
    # Get play-by-play data for the game
    pbp = playbyplayv2.PlayByPlayV2(game_id=game_id)

    # Parse play-by-play data
    play_by_play = pbp.get_data_frames()[0]

    # Print available columns to identify the relevant column for filtering
    # print("Available columns:", play_by_play.columns)

    # Convert 'PCTIMESTRING' to timedelta objects
    play_by_play['PCTIME'] = play_by_play['PCTIMESTRING'].apply(convert_to_timedelta)

    play_by_play['PCTIME']

    # Assuming the 'PCTIMESTRING' column represents the game clock
    # Filter events for the last two minutes of the game
    last_two_minutes = play_by_play[
        (play_by_play['PERIOD'] == 4) &  # Filter for the fourth quarter
        (play_by_play['PCTIME'] <= datetime.timedelta(minutes=2))  # Filter for the last two minutes
    ]

    last_two_min_idx = play_by_play[
        (play_by_play['PERIOD'] == 4) &  # Filter for the fourth quarter
        (play_by_play['PCTIME'] <= datetime.timedelta(minutes=2))  # Filter for the last two minutes
    ].index

    last_two_minute_index_plus1 = ([last_two_min_idx[0] - 1] + list(last_two_min_idx))

    last_two_minutes = play_by_play.loc[ last_two_minute_index_plus1]

    last_two_minutes



    if not last_two_minutes.empty:
        # Find the real-time when the game clock hits two minutes
        game_clock_at_two_minutes = last_two_minutes.iloc[0]['PCTIMESTRING']
        real_time_at_two_minutes = last_two_minutes.iloc[0]['WCTIMESTRING']
    else:
        game_clock_at_two_minutes = "No events recorded in the last two minutes"
        real_time_at_two_minutes = "Two minutes has not hti yet"

    # Find the real-time when the game ends
    game_end_time = last_two_minutes.iloc[-1]['PCTIMESTRING']
    real_time_at_end = last_two_minutes.iloc[-1]['WCTIMESTRING']

    # Convert game end time to datetime object
    game_end_datetime = datetime.datetime.strptime(game_end_time, "%M:%S").time()
    # realtime_at_two_min_datetime = datetime.datetime.strptime(real_time_at_two_minutes, "%M:%S").time()
    # realtime_end_datetime = datetime.datetime.strptime(real_time_at_end, "%M:%S").time()

    print("Real time when game clock hits two minutes:", real_time_at_two_minutes)
    print("Real time when game ends:", real_time_at_end)

    # Convert real time at two minutes and real time at end to datetime objects
    real_time_at_two_minutes_datetime = datetime.datetime.strptime(real_time_at_two_minutes, "%I:%M %p")
    real_time_at_end_datetime = datetime.datetime.strptime(real_time_at_end, "%I:%M %p")

    # Subtract the two times to get the duration
    real_time_duration = real_time_at_end_datetime - real_time_at_two_minutes_datetime

    print("Duration of real time between two minutes and game end:", real_time_duration)


    return real_time_duration


    


In [24]:
import pandas as pd

In [26]:
for game in games:
    display(pd.Series(game))

    time_last_two_minutes(game['gameId'])

    print('\n\n')

    

gameId             0022301070
homeTeamTricode           SAS
awayTeamTricode           NYK
dtype: object

Real time when game clock hits two minutes: 10:21 PM
Real time when game ends: 10:28 PM
Duration of real time between two minutes and game end: 0:07:00





gameId             0022301062
homeTeamTricode           CHA
awayTeamTricode           GSW
dtype: object

Real time when game clock hits two minutes: 9:11 PM
Real time when game ends: 9:14 PM
Duration of real time between two minutes and game end: 0:03:00





gameId             0022301063
homeTeamTricode           IND
awayTeamTricode           LAL
dtype: object

...

Real time when game clock hits two minutes: 9:49 PM
Real time when game ends: 10:01 PM
Duration of real time between two minutes and game end: 0:12:00





gameId             0022301068
homeTeamTricode           MIA
awayTeamTricode           POR
dtype: object

Real time when game clock hits two minutes: 10:07 PM
Real time when game ends: 10:09 PM
Duration of real time between two minutes and game end: 0:02:00





gameId             0022301069
homeTeamTricode           OKC
awayTeamTricode           PHX
dtype: object

Real time when game clock hits two minutes: 10:17 PM
Real time when game ends: 10:21 PM
Duration of real time between two minutes and game end: 0:04:00





In [9]:


# Game ID for the game you're interested in
# game_id = '0022301062'#'0022301065'




In [10]:
# # Let's inspect the DataFrame to find the appropriate column
# print(play_by_play.head())
# display(play_by_play.tail(10))

In [104]:
# type(play_by_play['PCTIMESTRING'][0])

str

Duration of real time between two minutes and game end: 0:03:00
